In [62]:
import random
import warnings
import numpy as np
import matplotlib.pyplot as plt
import dataLoader as dl
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import util as ut

#from Inner_Speech_Dataset.Plotting.ERPs import 
from Inner_Speech_Dataset.Python_Processing.Data_extractions import  Extract_data_from_subject
from Inner_Speech_Dataset.Python_Processing.Data_processing import  Select_time_window, Transform_for_classificator, Split_trial_in_time
from Inner_Speech_Dataset.Python_Processing.Data_processing import  Calculate_power_windowed
from Inner_Speech_Dataset.Python_Processing.Utilitys import picks_from_channels
from Inner_Speech_Dataset.Python_Processing.Data_processing import Average_in_frec

In [63]:


sampling_rate = 256
nr_of_datasets=1
data, labels = dl.load_multiple_datasets(nr_of_datasets=nr_of_datasets, sampling_rate=sampling_rate, t_min=2, t_max=3, twoDLabels=True)

ch_names = ut.get_channelNames()

print(data.shape)
data_p =  ut.get_power_array(data[:,:128,:], sampling_rate, trialSplit=1).squeeze()
print(data_p.shape)
#print(data_p[:,:,1])

#Getting Freq Data 
nr_of_buckets = 15
buckets = ut.createFreqBuckets(data[:,:128,:], nr_of_buckets)

#buckets = np.reshape(np.linspace(0,50,nr_of_buckets*2),[nr_of_buckets, -1])
#buckets = np.array([[0,3],[4,8],[9,15],[16,34],[35,45],[45,80]])
nr_of_buckets = buckets.shape[0]
print("buckets")
print(buckets)
data_f = ut.data_into_freq_buckets(data[:,:128,:], nr_of_buckets, buckets)
data_f = data_f[:,:, [11 , 6 , 2 , 4]]
#data_f = np.concatenate([dataf, np.zeros([data_f.shape[0], nr_of_datasets, data_f.shape[2]])], axis=1)

print(data_f.shape)


## Normalize data



Data shape: [trials x channels x samples]
(500, 128, 256)
Labels shape
(500, 4)
Final data shape
(100, 128, 256)
Final labels shape
(100,)
Up is 0.0 and Down is 1.0
(100, 128, 256)
(100, 128, 2)
buckets
[[  0   2]
 [  2   5]
 [  5   8]
 [  8  12]
 [ 12  16]
 [ 16  21]
 [ 21  28]
 [ 28  36]
 [ 36  47]
 [ 47  60]
 [ 60  77]
 [ 76  94]
 [ 91 112]
 [ 67  79]
 [  3   4]]
(100, 128, 4)


In [64]:
#from scipy.fft import rfft, ifft, fftshift, fftfreq
#fftfreq(data.shape[2],1/data.shape[2])

In [65]:
print(data_f.shape)
print(data_p.shape)
print(labels.shape)
print(data_p[2,5])

data = np.concatenate([data_f, data_p], axis =2 ) #, data
print(data.shape)
data = keras.utils.normalize(data, axis=1, order=2)
print(data.shape)
#print(data[2,5])
#print(labels[2])
print(labels[:].shape)

(100, 128, 4)
(100, 128, 2)
(100, 2)
[9.56732240e-11 4.90024352e-11]
(100, 128, 6)
(100, 128, 6)
(100, 2)


In [66]:
#Add end of ekg with specific array for each person. 


In [67]:
#Splitting into training and test data
#print(labels)


order = np.arange(labels.shape[0])
np.random.shuffle(order)

temp_data = np.zeros(data.shape)
temp_labels = np.zeros(labels.shape)

for x in range(labels.shape[0]):
    i = order[x]
    
    temp_data[x] = data[i]
    temp_labels[x] = labels[i]

data = temp_data
labels = temp_labels

data_train, data_test = np.split(data, indices_or_sections=[int(labels.shape[0]*0.75)],axis=0)
labels_train, labels_test = np.split(labels, indices_or_sections=[int(labels.shape[0]*0.75)],axis=0)
print(labels_train.shape)
print(data_train.shape)
print(data_test.shape)




(75, 2)
(75, 128, 6)
(25, 128, 6)


In [68]:
# #!pip3 install sklearn -q
# from tabnanny import verbose
# from sklearn.svm import SVC
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
# from sklearn.feature_selection import SelectKBest, f_classif
# from sklearn.metrics import classification_report

# def svmPipeline(data_train, data_test, labels_train,
#     labels_test, kernel="linear", degree=3, gamma="auto", C = 1
#     , coefs = np.zeros([1, data_train.shape[1]*data_train.shape[2]])):
    
#     anova_filter = SelectKBest(f_classif, k=25)
#     clf = make_pipeline(anova_filter, StandardScaler() ,  SVC(
#         gamma=gamma, kernel=kernel, degree=degree, verbose=False, C=C))

#     clf.fit(np.reshape(data_train, [data_train.shape[0], -1]), labels_train)
#     predictions = clf.predict(np.reshape(data_test, [data_test.shape[0], -1]))
    
#     print(classification_report(labels_test, predictions))
#     if kernel == "linear":
#         coefs = coefs + clf[:-1].inverse_transform(clf[-1].coef_)
#     #print(clf[:-1].inverse_transform(clf[-1].coef_).shape)

#     correct = np.zeros(labels_test.shape)
#     correctamount=0
#     for nr, pred in enumerate(predictions,0):
#         if pred == labels_test[nr]:
#             correct[nr] = 1
#             correctamount +=1


#     return correctamount/labels_test.shape[0], coefs

# #testing using different kernels, C and degrees. 
# coefs = np.zeros([1, data_train.shape[1]*data_train.shape[2]])
# for kernel in ["linear", "rbf", "sigmoid"]:

#     for C in np.linspace(0.2,1,5):
    
#         if kernel == "linear":
#             for degree in range(1,3):
#                 res, coefs = svmPipeline(data_train, data_test, labels_train, 
#                 labels_test, degree=degree, kernel = kernel, C = C, coefs=coefs)
#                 print("Result for degree {}, kernel {}, C = {}: {}".format(degree, kernel, C,  res))
#         # else:
#         #     for gamma in ["auto", "scale"]:

#         #         res = svmPipeline(data_train, data_test, labels_train, 
#         #         labels_test, degree=degree, kernel = kernel, gamma=gamma, C = C)
#         #         print("Result for gamma {}, kernel {}, C = {}: {}".format(gamma, kernel, C, res))

# coefs = np.reshape(coefs, [128,-1])

In [69]:
# #Checks which frequency buckets are most valuable
# def coefFeatureCheck(coefs, buckets):
#     for x in range(buckets.shape[0], buckets.shape[0] + 2):
#         print("Power feature bin {} has value {} ".format(x-buckets.shape[0], np.sum(coefs[:,x])))

#     freqBins = np.zeros(buckets.shape[0])
#     for x in range(0,buckets.shape[0]):
#         # #print(np.sum(coefs[:,0]))
#         freqBins [x] = np.sum(coefs[:,x])
#         print("Frequency bin {} has feature value of {}".format(buckets[x], np.sum(coefs[:,x])))

#     channelBins = np.zeros(coefs.shape[0])
#     for x in range(0, coefs.shape[0]):
#         #print(np.sum(coefs[:,0]))
#         if  np.sum(coefs[x,:]) == 0:
#             continue
#         channelBins[x] = np.sum(coefs[x,:])
#         print("Channel bin {} has feature value of {}".format(x, np.sum(coefs[x,:])))

#     #print(channelBins)
#     #print(freqBins)
#     print(buckets[np.argpartition(freqBins, -4)[-4:]])
#     return np.argpartition(freqBins, -4)[-4:]
# mostUsefulFreqBuckets =  coefFeatureCheck(coefs, buckets)
# print(mostUsefulFreqBuckets)



In [87]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.keras.backend.clear_session()





eeg_model = tf.keras.Sequential([

    layers.LocallyConnected1D(500, input_shape = (data_train.shape[1],data_train.shape[2]), 
     kernel_size=15, 
     padding="valid", strides=8,
     activation="relu"),

    layers.Dropout(0.7),
    
    layers.LocallyConnected1D(500, 
    kernel_size=15, 
    padding="valid", strides=5,
    activation="relu"),
    
    layers.Dropout(0.7),
    
    layers.Dense(units=200, activation="relu"),
    
    layers.Dropout(0.7),

    layers.Flatten(),

    layers.Dense(units=2, activation="softmax")


])
eeg_model.build()
eeg_model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 locally_connected1d (Locall  (None, 15, 500)          682500    
 yConnected1D)                                                   
                                                                 
 dropout (Dropout)           (None, 15, 500)           0         
                                                                 
 locally_connected1d_1 (Loca  (None, 1, 500)           3750500   
 llyConnected1D)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 1, 500)            0         
                                                                 
 dense (Dense)               (None, 1, 200)            100200    
                                                                 
 dropout_2 (Dropout)         (None, 1, 200)            0

In [88]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
#callback2 = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)
#tensorboard_callback = tf.keras.callbacks.TensorBoard(histogram_freq=2)

eeg_model.compile(optimizer='adam',
              loss="binary_crossentropy",
              metrics=['accuracy'])

data_train_send = np.reshape(data_train, [data_train.shape[0], 1 , data_train.shape[1], data_train.shape[2], 1])
#data_train_send = np.reshape(data_train, [data_train.shape[0], 1 , data_train.shape[1], data_train.shape[2], 1])

data_test_send = np.reshape(data_test, [data_test.shape[0], 1 , data_test.shape[1], data_test.shape[2], 1])
#data_test_send = np.reshape(data_test, [data_test.shape[0], 1 , data_test.shape[1], data_test.shape[2], 1])

outputs = eeg_model.fit(data_train, labels_train, validation_split=0.2 ,callbacks=[], epochs=200) #

print("Results")
eeg_model.evaluate(data_test, labels_test)
result = eeg_model.predict(data_test)

Epoch 1/200
2/2 [==============================] - 1s 225ms/step - loss: 0.6925 - accuracy: 0.5167 - val_loss: 0.6925 - val_accuracy: 0.5333
Epoch 2/200
2/2 [==============================] - 0s 37ms/step - loss: 0.7056 - accuracy: 0.4167 - val_loss: 0.6922 - val_accuracy: 0.5333
Epoch 3/200
2/2 [==============================] - 0s 43ms/step - loss: 0.6979 - accuracy: 0.5833 - val_loss: 0.6921 - val_accuracy: 0.5333
Epoch 4/200
2/2 [==============================] - 0s 38ms/step - loss: 0.7026 - accuracy: 0.5667 - val_loss: 0.6922 - val_accuracy: 0.5333
Epoch 5/200
2/2 [==============================] - 0s 37ms/step - loss: 0.6967 - accuracy: 0.5333 - val_loss: 0.6928 - val_accuracy: 0.5333
Epoch 6/200
2/2 [==============================] - 0s 43ms/step - loss: 0.6830 - accuracy: 0.5833 - val_loss: 0.6926 - val_accuracy: 0.5333
Epoch 7/200
2/2 [==============================] - 0s 37ms/step - loss: 0.7575 - accuracy: 0.3500 - val_loss: 0.6920 - val_accuracy: 0.5333
Epoch 8/200
2/2 [==

In [72]:

eeg_model.save('saved_model/my_model6')

INFO:tensorflow:Assets written to: saved_model/my_model6\assets


In [73]:
print("Results")
eeg_model.evaluate(data_test, labels_test)
result = eeg_model.predict(data_test)

result2 = []
print(result)
for res in result:
    x = np.maximum(res[0], res[1])
    result2.append(int(np.where(res == x)[0]))

result2 = np.array(result2)
print(result2)

Results
1/1 [==============================] - 0s 15ms/step
[[1.2240368e-03 9.9877590e-01]
 [2.8045848e-04 9.9971956e-01]
 [4.4816053e-03 9.9551833e-01]
 [1.1990033e-05 9.9998796e-01]
 [9.7778147e-01 2.2218565e-02]
 [6.2170619e-01 3.7829375e-01]
 [2.9595515e-02 9.7040445e-01]
 [1.0000000e+00 3.2494463e-10]
 [7.3025309e-05 9.9992692e-01]
 [3.7249851e-05 9.9996281e-01]
 [7.6678523e-04 9.9923325e-01]
 [6.8873994e-04 9.9931121e-01]
 [4.4097304e-09 1.0000000e+00]
 [9.8012882e-01 1.9871198e-02]
 [8.1294209e-01 1.8705791e-01]
 [5.6858062e-05 9.9994314e-01]
 [6.2121862e-01 3.7878144e-01]
 [9.9807501e-01 1.9249528e-03]
 [1.3794781e-03 9.9862051e-01]
 [9.9910147e-02 9.0008986e-01]
 [4.3577966e-04 9.9956423e-01]
 [9.8522121e-01 1.4778783e-02]
 [9.9989581e-01 1.0418362e-04]
 [1.2996230e-05 9.9998701e-01]
 [1.3757840e-02 9.8624218e-01]]
[1 1 1 1 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 0 0 1 1]
